In [ ]:
#hide
from eptune import *
%pylab inline

# eptune

> eptune (evolutionary parameter tuning) is a python package trying to use evolutionary computation algorithms to do parameter tuning.

![CI](https://github.com/wanglongqi/eptune/workflows/CI/badge.svg)

![Logo](https://github.com/wanglongqi/eptune/raw/master/nbs/eptune.png)

## Install

`pip install eptune`

## How to use

Using following lines can fine tune MNIST dataset with 5-Fold CV performance using the `qtuneSimple` function.

In [ ]:
from eptune.sample_cases import DigitsCV
from eptune.quick import qtuneSimple
from eptune.parameter import *
from sklearn.svm import SVC

# Prameter space to search
params = [
    LogFloatParameter([0.01, 1e4], 'C'),
    CategoricalParameter(['rbf'], 'kernel'),
    LogFloatParameter([1e-6, 1e4], 'gamma')
]

# Define objective function
cv_svc_digits = DigitsCV(SVC())


def evaluate(params):
    return cv_svc_digits.cv_loss_with_params(cv=5, **params)


# Call `qtuneSimple`
population, logbook, hof = qtuneSimple(params,
                                       evaluate,
                                       n_pop=10,
                                       n_jobs=10,
                                       mutpb=0.6,
                                       cxpb=0.8,
                                       seed=42)

# Plot the logbook if needed
fig = logbook.plot(['min', 'avg'])

The best parameters are stored in `HallofFame` object:

In [ ]:
hof

## Iterator Interface

An iterator interface, `qtuneIterate`, is provided in the `quick` module, so that one can treat the optimization procedure as a iterative process. When the iterator exhausted, the optimization process is ended. Following is an example:

In [ ]:
from eptune.quick import qtuneIterate
from eptune.parameter import *
# Prameter space to search
params = [
    LogFloatParameter([0.01, 1e4], 'C'),
    CategoricalParameter(['rbf'], 'kernel'),
    LogFloatParameter([1e-6, 1e4], 'gamma')
]
qi = qtuneIterate(params, n_pop=10, mutpb=0.6, cxpb=0.8, seed=42)

Directly call this class will return an object that can be used as an iterator.

In [ ]:
iterator = qi()

For each iteration, the iterator will return two elements. The first element is the parameters to be used to in the estimator or function, the other element is the Condition object used in `qtuneIterate.set_result` function.

In [ ]:
p, c = next(iterator)
p

One can do what ever you want here, such as query database, download web page, or even ask for user's input. When all the things necessary are done, one need to call `qtuneIterate.set_result` to return the control to the package and generate next parameters for next iteration.

In this example, the same function in above example is used to get the loss of the estimator with above parameters.

In [ ]:
from eptune.sample_cases import DigitsCV
from sklearn.svm import SVC
# Define objective function
cv_svc_digits = DigitsCV(SVC())


def evaluate(params):
    return cv_svc_digits.cv_loss_with_params(cv=5, **params)

loss = evaluate(p)
loss

After we get the loss value, we can use `qtuneIterate.set_result` to return the control to the package as follows:

In [ ]:
qi.set_result(loss, c)

In [ ]:
p, c = next(iterator)
p

Repeat this process until the iterator is exhausted, then the optimization is done. In practice, you can simply use a for loop to do the job.

In [ ]:
qi = qtuneIterate(params, n_pop=10, mutpb=0.6, cxpb=0.8, ngen=3, seed=42)
for p, c in qi():
    print(f'Calculating loss for {p}')
    loss = evaluate(p)
    print(f'Loss is {loss[0]}')
    qi.set_result(loss, c)

After the iteration, the `qtuneIterate` instance store the results in itself.

In [ ]:
qi.population

In [ ]:
qi.hof

In [ ]:
fig = qi.logbook.plot(['min', 'avg'])

## More control

If you want more control, you can check:
1. `eptune.sklearn` module provides `ScikitLearner` or `ScikitLearnerCV` for fine tune parameter of estimators with scikit learn API. Examples are also provided in the documentation.
2. `eptune.algorithms` module provides algorithms to access the DEAP framework directly.